In [1]:
import dask # get error with xr.open without
import earthaccess
import glob
import h5netcdf
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import xarray as xr

/work/pi_cjgleason_umass_edu/.conda/envs/narrowPIXC/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### SWOT PIXC: 298_080L

In [2]:
auth = earthaccess.login()

In [8]:
pixc_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_PIXC_2.0', 
                                        granule_name = '*530_013_233L_*') # Willamette 

In [9]:
pixc_results

[Collection: {'Version': '2.0', 'ShortName': 'SWOT_L2_HR_PIXC_2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -123.64687248122013, 'SouthBoundingCoordinate': 44.29428314992776, 'EastBoundingCoordinate': -122.57508310107417, 'NorthBoundingCoordinate': 45.062543416912895}]}, 'Track': {'Cycle': 530, 'Passes': [{'Pass': 13, 'Tiles': ['233L']}]}}}
 Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2023-05-24T00:09:12.823Z', 'BeginningDateTime': '2023-05-24T00:09:01.798Z'}}
 Size(MB): 531.6269626617432
 Data: ['https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_PIXC_2.0/SWOT_L2_HR_PIXC_530_013_233L_20230524T000901_20230524T000912_PGC0_01.nc']]

In [3]:
filepath = '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_011_298_080L_20240225T115735_20240225T115746_PIC0_02.nc'

##### Global

In [6]:
# load global attributes for tile
ds_global = xr.open_mfdataset(filepath, engine='h5netcdf')

In [7]:
ds_global

<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/65)
    Conventions:                                            CF-1.7
    title:                                                  Level 2 KaRIn Hig...
    institution:                                            CNES
    source:                                                 Level 1B KaRIn Hi...
    history:                                                2024-02-29T09:57:...
    platform:                                               SWOT
    ...                                                     ...
    xref_pole_location_file:                                SMM_PO1_AXXCNE202...
    xref_geco_database_version:                             v102
    ellipsoid_semi_major_axis:                              6378137.0
    ellipsoid_flattening:                                   0.003352810664747...
    references:                                             1.2
    product_version:                                        02

In [45]:
along_track = ds_global.slc_along_track_resolution

In [5]:
# Get bounding coordinates for SWOT tile
west_lon = ds_global.geospatial_lon_min
south_lat = ds_global.geospatial_lat_min
east_lon = ds_global.geospatial_lon_max
north_lat = ds_global.geospatial_lat_max

In [9]:
def toslant(pixc, key='height'):
    az = pixc.azimuth_index.astype(int)
    rng = pixc.range_index.astype(int)
    out = np.zeros((pixc.interferogram_size_azimuth + 1, pixc.interferogram_size_range + 1)) + np.nan
    var = pixc[key]
    out[az, rng] = var
    return out

##### Point Cloud

In [3]:
# load pixel cloud
ds = xr.open_mfdataset(filepath, group = 'pixel_cloud', engine='h5netcdf')

In [4]:
ds

<xarray.Dataset> Size: 2GB
Dimensions:                                (points: 5584058, complex_depth: 2,
                                            num_pixc_lines: 3266)
Coordinates:
    latitude                               (points) float64 45MB dask.array<chunksize=(507642,), meta=np.ndarray>
    longitude                              (points) float64 45MB dask.array<chunksize=(507642,), meta=np.ndarray>
Dimensions without coordinates: points, complex_depth, num_pixc_lines
Data variables: (12/61)
    azimuth_index                          (points) float64 45MB dask.array<chunksize=(930677,), meta=np.ndarray>
    range_index                            (points) float64 45MB dask.array<chunksize=(930677,), meta=np.ndarray>
    interferogram                          (points, complex_depth) float32 45MB dask.array<chunksize=(1396015, 1), meta=np.ndarray>
    power_plus_y                           (points) float32 22MB dask.array<chunksize=(930677,), meta=np.ndarray>
    power_minus_y                          (points) float32 22MB dask.array<chunksize=(930677,), meta=np.ndarray>
    coherent_power                         (points) float32 22MB dask.array<chunksize=(930677,), meta=np.ndarray>
    ...                                     ...
    pixc_line_qual                         (num_pixc_lines) float64 26kB dask.array<chunksize=(3266,), meta=np.ndarray>
    pixc_line_to_tvp                       (num_pixc_lines) float32 13kB dask.array<chunksize=(3266,), meta=np.ndarray>
    data_window_first_valid                (num_pixc_lines) float64 26kB dask.array<chunksize=(3266,), meta=np.ndarray>
    data_window_last_valid                 (num_pixc_lines) float64 26kB dask.array<chunksize=(3266,), meta=np.ndarray>
    data_window_first_cross_track          (num_pixc_lines) float32 13kB dask.array<chunksize=(3266,), meta=np.ndarray>
    data_window_last_cross_track           (num_pixc_lines) float32 13kB dask.array<chunksize=(3266,), meta=np.ndarray>
Attributes:
    description:                 cloud of geolocated interferogram pixels
    interferogram_size_azimuth:  3266
    interferogram_size_range:    4696
    looks_to_efflooks:           1.544884945080654
    num_azimuth_looks:           7.0
    azimuth_offset:              3

In [ ]:
test = ds.to_dataframe()